# BULK

In [1]:
import ase
from ase.db import connect
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.eos import calculate_eos

In [2]:
atom = bulk('Al', 'fcc')
atom.get_volume()

16.60753125

In [3]:
atom = bulk('Cu', 'fcc')
atom.calc =EMT()
atom.get_volume(), atom.get_potential_energy() # 想要获得能量就得指定calc方法

(11.76147025, -0.005681511358588409)

In [4]:
eos = calculate_eos(atom)
v,e,B = eos.fit() # 得到energy最小值 -- 原子的相对位置不变
v,e,B

(11.565374510214, -0.007036478731361484, 0.8392875566841875)

In [5]:
atom.cell *= (v / atom.get_volume()) ** (1/3)
atom.get_potential_energy()

-0.0070364920483623195

In [6]:
# ase.io.write("POSCAR",atom,format="vasp")

In [7]:
db = ase.db.connect('bulk.db')
for atom in ['Al', 'Ni', 'Cu', 'Pd', 'Ag', 'Pt', 'Au']:
    atoms = bulk(atom, 'fcc')
    atoms.calc = EMT()
    eos = ase.eos.calculate_eos(atoms)
    v,e,B = eos.fit()
    atoms.cell *= (v/atoms.get_volume()) ** (1/3)
    atoms.get_potential_energy()
    db.write(atoms, bm=B) # bm: bulk modulus

# ADSORBATES

## 引入1 - fcc111

In [9]:
from ase.build import fcc111
slab = ase.build.fcc111("Al", size=(2,2,3), vacuum=10, periodic=False)
ase.build.add_adsorbate(slab, 'H', 1.5, 'ontop')
slab.center(vacuum=10.0, axis=2)
# ase.io.write("POSCAR1", slab, format='vasp')
print(slab.get_tags())
len(slab)

[3 3 3 3 2 2 2 2 1 1 1 1 0]


13

## 引入2 - BFGS
BFGS 只优化原子位置，不优化胞

In [12]:
from ase.optimize import BFGS
from ase.constraints import FixAtoms

In [11]:
h2o = ase.Atoms("H2O", positions=[(0,0,0), (1,0,0), (0,1,0)], calculator=EMT())
bfgs = BFGS(h2o, trajectory="h2o.traj", restart="qn.pckl")
atoms = bfgs.run(fmax=0.1, steps=100) # fmax: 收敛精度

      Step     Time          Energy         fmax
BFGS:    0 17:58:15        2.727814        3.7034
BFGS:    1 17:58:15        2.175891        2.8245
BFGS:    2 17:58:15        2.014597        2.5453
BFGS:    3 17:58:15        1.923292        1.2353
BFGS:    4 17:58:15        1.887674        0.3347
BFGS:    5 17:58:15        1.883974        0.2044
BFGS:    6 17:58:15        1.882735        0.1729
BFGS:    7 17:58:15        1.879417        0.0947


In [12]:
symb="Al"
ads="H"
n=3 # z轴上选取三层
atoms = fcc111(symb, (1, 1, n))
ase.build.add_adsorbate(atoms, ads, height=1.0, position='fcc')
# print(atoms.get_tags()) # [3,2,1,0]
# Constrain all atoms except the adsorbate:
# print(len(atoms)) # 4
fixed = list(range(len(atoms) - 1))
# print(fixed) # [0,1,2]
# print(FixAtoms(indices=fixed))
atoms.constraints = FixAtoms(indices=fixed)
atoms.calc = EMT()
print(atoms.get_cell())
opt = BFGS(atoms, logfile="out.log")
opt.run(fmax=0.01)
print(atoms.get_cell())

Cell([[2.8637824638055176, 0.0, 0.0], [1.4318912319027588, 2.4801083645679673, 0.0], [0.0, 0.0, 0.0]])
Cell([[2.8637824638055176, 0.0, 0.0], [1.4318912319027588, 2.4801083645679673, 0.0], [0.0, 0.0, 0.0]])


## 实例

### 补充

In [25]:

db1 = ase.db.connect("bulk.db")
db2 = ase.db.connect("ads.db")

In [26]:
iter_ = iter(db.select())
dst = next(iter_)
dir(dst)[34:]

['bm',
 'calculator',
 'calculator_parameters',
 'cell',
 'charge',
 'constrained_forces',
 'constraints',
 'count_atoms',
 'ctime',
 'data',
 'energy',
 'fmax',
 'forces',
 'formula',
 'get',
 'id',
 'key_value_pairs',
 'mass',
 'mtime',
 'natoms',
 'numbers',
 'pbc',
 'positions',
 'smax',
 'symbols',
 'toatoms',
 'unique_id',
 'user',
 'volume']

In [27]:
dst. symbols, dst.cell

(['Al'],
 array([[0.        , 1.99715944, 1.99715944],
        [1.99715944, 0.        , 1.99715944],
        [1.99715944, 1.99715944, 0.        ]]))

### 计算吸附物能量

In [28]:
def run(name, a, layer, ads):
    atoms = ase.build.fcc111(name, (1,1,n), a=a)
    ase.build.add_adsorbate(atoms, ads, height=1, position="fcc")

    # fix atoms except adsorbs
    fixed = list(range(len(atoms) - 1))
    atoms.constraints = ase.constraints.FixAtoms(indices = fixed)

    atoms.calc = EMT()

    opt = ase.optimize.BFGS(atoms, logfile=None)
    opt.run(fmax=0.01)
    return atoms

for atom in db1.select():
    a = atom.cell[0,1] * 2
    symbol = atom.symbols[0]
    for n in [1,2,3]:
        for ads in "CNO":
            id = db2.reserve(layers=n, surf=symbol, ads=ads) # 创建对应参数的row， 如果存在，id=None
            if id is not None:
                atoms = run(symbol, a, n, ads)
                db2.write(atoms, surf=symbol, layers=n, ads=ads)
                del db2[id]

### 计算纯净的表面的能量

In [30]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.db import connect
from ase.build import fcc111

db1 = connect('bulk.db')
db2 = connect('ads.db')


def run(symb, a, n):
    atoms = fcc111(symb, (1, 1, n), a=a)
    atoms.calc = EMT()
    atoms.get_forces()
    return atoms


# Clean slabs:
for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        id = db2.reserve(layers=n, surf=symb, ads='clean')
        if id is not None:
            atoms = run(symb, a, n)
            db2.write(atoms, id=id, layers=n, surf=symb, ads='clean')

# Atoms:
for ads in 'CNO':
    a = Atoms(ads)
    a.calc = EMT()
    a.get_potential_energy()
    db2.write(a)

"""
Say we want those 24 reference energies (clean surfaces and isolated adsorbates) in a refs.db file instead of the big ads.db file. We could change the refs.py script and run the calculations again, but we can also manipulate the files using the ase db tool. First, we move over the clean surfaces:

$ ase db ads.db ads=clean --insert-into refs.db
Added 0 key-value pairs (0 pairs updated)
Inserted 21 rows
$ ase db ads.db ads=clean --delete --yes
Deleted 21 rows

and then the three atoms (pbc=FFF, no periodicity):

$ ase db ads.db pbc=FFF --insert-into refs.db
Added 0 key-value pairs (0 pairs updated)
Inserted 3 rows
$ ase db ads.db pbc=FFF --delete --yes
Deleted 3 rows
$ ase db ads.db -n
63 rows
$ ase db refs.db -n
24 rows
"""

### 分析

In [32]:
refs = connect('refs.db')
db = connect('ads.db')

for row in db.select():
    ea = row.energy - refs.get(formula=row.ads).energy - refs.get(layers=row.layers, surf=row.surf).energy
    h = row.positions[-1,2] - row.positions[-2,2]
    db.update(row.id, height=h, ea=ea)